In [140]:
import pandas as pd
import os
import numpy as np

%reload_ext autoreload
%autoreload 2

## Residential (SF)

### Create df with hourly LADWP peak pricing periods (low peak, high peak, base) in 2019. 

##### Note: The year 2019 starts on a Tuesday. If running a different year of Harbor data, need to adjust pricing periods to start on the correct day of the week.

In [141]:
#Annie constructed this csv based on LADWP retail rate pricing periods for each day and hour of the week. 
peak_periods = pd.read_csv('data/peak_periods_2019.csv')

In [142]:
#Expand to full year.
almost_full_year = pd.concat([peak_periods]*52, ignore_index=True)
last_hours = peak_periods[0:24]
full_year = pd.concat([almost_full_year, last_hours], ignore_index=True)

In [143]:
#Label each hour in the year with its corresponding seasonal pricing period (by month).

jan_march_days = 90
jan_march_hour_start = 0
jan_march_hour_end = jan_march_days*24-1

apr_may_days = 61
apr_may_hour_start = jan_march_hour_end + 1
apr_may_hour_end = apr_may_hour_start + (apr_may_days*24) -1

jun_days = 30
jun_hour_start = apr_may_hour_end + 1
jun_hour_end = jun_hour_start + (jun_days*24) -1

jul_sept_days = 92
jul_sept_hour_start = jun_hour_end + 1
jul_sept_hour_end = jul_sept_hour_start + (jul_sept_days*24) -1

oct_dec_days = 92
oct_dec_hour_start = jul_sept_hour_end + 1
oct_dec_hour_end = oct_dec_hour_start + (oct_dec_days*24) -1

full_year.loc[jan_march_hour_start:jan_march_hour_end,'month'] = 'jan_mar'
full_year.loc[apr_may_hour_start:apr_may_hour_end,'month'] = 'apr_may'
full_year.loc[jun_hour_start:jun_hour_end,'month'] = 'jun'
full_year.loc[jul_sept_hour_start:jul_sept_hour_end,'month'] = 'jul_sept'
full_year.loc[oct_dec_hour_start:oct_dec_hour_end,'month'] = 'oct_dec'

### Read in LADWP residential retail TOU rate table and assign rates to each hour in year 2019.

In [144]:
res_tou_rate_lookup = pd.read_csv('data/res_retail_tou_rate_period_lookup.csv')

In [145]:
month_period = full_year.groupby(['month', 'period'])

In [146]:
res_hourly_tou_rate = pd.DataFrame()

for name, group in month_period:
    month = name[0]
    period = name[1]
    month_inds = res_tou_rate_lookup['month'] == month
    rate = res_tou_rate_lookup.loc[month_inds, period].item()
    #Convert 2021$ to 2018$ (0.96/1) and mulitply by 1000 to get price per MWh.
    group['rate'] = rate * 0.96 * 1000

    res_hourly_tou_rate = pd.concat([res_hourly_tou_rate, group])
    
res_hourly_tou_rate['units'] = '2018$/MWh'
res_hourly_tou_rate = res_hourly_tou_rate.sort_index()

In [147]:
res_hourly_tou_rate.to_csv('data/ladwp_res_hourly_tou_rate.csv')

## Commercial & Residential (MF)

#### Get full_year df from above and adjust peak pricing periods for commercial sector.

In [148]:
jan_may = ['jan_mar','apr_may']
jan_may_inds = full_year['month'].isin(jan_may)
full_year.loc[jan_may_inds, 'month_comm'] = 'jan_may'

jun_sept = ['jun','jul_sept']
jun_sept_inds = full_year['month'].isin(jun_sept)
full_year.loc[jun_sept_inds, 'month_comm'] = 'jun_sept'

oct_dec_inds = full_year['month']=='oct_dec'
full_year.loc[oct_dec_inds, 'month_comm'] = 'oct_dec'

In [149]:
month_period_comm = full_year.groupby(['month_comm', 'period'])

#### Read in commercial TOU rate.

In [150]:
comm_tou_rate_lookup = pd.read_csv('data/comm_retail_rate_tou_lookup.csv')

In [151]:
comm_tou_rate_lookup

,rate,month,demand,high,low,base
0,A1,jan_may,5.36,0.10000,0.10000,0.07082
1,A1,jun_sept,5.36,0.14990,0.10000,0.07082
2,A1,oct_dec,5.36,0.10000,0.10000,0.07082
3,A2,jan_may,5.36,0.05688,0.05688,0.03895
4,A2,jun_sept,5.36,0.06322,0.05595,0.03522
5,A2,oct_dec,5.36,0.05688,0.05688,0.03895


##### Add in monthly demand charge rates for rate A-2.

In [152]:
# comm_tou_rate_lookup['demand_monthly_high_peak'] = 0
# comm_tou_rate_lookup['demand_monthly_low_peak'] = 0
a2_inds = comm_tou_rate_lookup['rate']=='A2'
jan_may_inds = comm_tou_rate_lookup['month']=='jan_may'
jun_sept_inds = comm_tou_rate_lookup['month']=='jun_sept'
oct_dec_inds = comm_tou_rate_lookup['month']=='oct_dec'

# comm_tou_rate_lookup.loc[a2_inds & jan_may_inds, 'demand_monthly_high_peak'] = 4.75
# comm_tou_rate_lookup.loc[a2_inds & jun_sept_inds, 'demand_monthly_high_peak'] = 10
# comm_tou_rate_lookup.loc[a2_inds & oct_dec_inds, 'demand_monthly_high_peak'] = 4.75

# comm_tou_rate_lookup.loc[a2_inds & jun_sept_inds, 'demand_monthly_low_peak'] = 3.75

In [153]:
comm_tou_rate_lookup

,rate,month,demand,high,low,base
0,A1,jan_may,5.36,0.10000,0.10000,0.07082
1,A1,jun_sept,5.36,0.14990,0.10000,0.07082
2,A1,oct_dec,5.36,0.10000,0.10000,0.07082
3,A2,jan_may,5.36,0.05688,0.05688,0.03895
4,A2,jun_sept,5.36,0.06322,0.05595,0.03522
5,A2,oct_dec,5.36,0.05688,0.05688,0.03895


In [154]:
comm_hourly_tou_rate = pd.DataFrame()

for name, group in month_period_comm:
    month_comm = name[0]
    period = name[1]
    month_comm_inds = comm_tou_rate_lookup['month'] == month_comm
    
    small_comm_rate_inds = comm_tou_rate_lookup['rate'] == 'A1'
    large_comm_rate_inds = comm_tou_rate_lookup['rate'] == 'A2'
    
    small_comm_rate = comm_tou_rate_lookup.loc[month_comm_inds & small_comm_rate_inds, period].item()
    large_comm_rate = comm_tou_rate_lookup.loc[month_comm_inds & large_comm_rate_inds, period].item()
    
#     demand_charge = comm_tou_rate_lookup.loc[month_comm_inds & small_comm_rate_inds, 'demand'].item()
#     demand_monthly_high_charge = comm_tou_rate_lookup.loc[month_comm_inds & large_comm_rate_inds, 'demand_monthly_high_peak'].item()
#     demand_monthly_low_charge = comm_tou_rate_lookup.loc[month_comm_inds & large_comm_rate_inds, 'demand_monthly_low_peak'].item()
    
    #Convert 2021$ to 2018$ (0.96/1) and mulitply by 1000 to get price per MWh or MW.
    group['rate_small_comm'] = small_comm_rate * 0.96 * 1000
    group['rate_large_comm'] = large_comm_rate * 0.96 * 1000
#     group['demand_charge_yearly_peak_per_mw'] = demand_charge * 0.96 * 1000
#     group['demand_charge_monthly_high_peak_per_mw'] = demand_monthly_high_charge * 0.96 * 1000
#     group['demand_charge_monthly_low_peak_per_mw'] = demand_monthly_low_charge * 0.96 * 1000

    comm_hourly_tou_rate = pd.concat([comm_hourly_tou_rate, group])
    
comm_hourly_tou_rate['units'] = '2018$/MWh'
comm_hourly_tou_rate = comm_hourly_tou_rate.sort_index()

In [155]:
comm_hourly_tou_rate

,day,hour_starting,period,month,month_comm,rate_small_comm,rate_large_comm,units
0,tues,0,base,jan_mar,jan_may,67.9872,37.3920,2018$/MWh
1,tues,1,base,jan_mar,jan_may,67.9872,37.3920,2018$/MWh
2,tues,2,base,jan_mar,jan_may,67.9872,37.3920,2018$/MWh
3,tues,3,base,jan_mar,jan_may,67.9872,37.3920,2018$/MWh
4,tues,4,base,jan_mar,jan_may,67.9872,37.3920,2018$/MWh
...,...,...,...,...,...,...,...,...
8755,tues,19,low,oct_dec,oct_dec,96.0000,54.6048,2018$/MWh
8756,tues,20,base,oct_dec,oct_dec,67.9872,37.3920,2018$/MWh
8757,tues,21,base,oct_dec,oct_dec,67.9872,37.3920,2018$/MWh
8758,tues,22,base,oct_dec,oct_dec,67.9872,37.3920,2018$/MWh


In [156]:
comm_hourly_tou_rate.to_csv('data/ladwp_comm_hourly_tou_rate.csv')

#### Read in demand charge rates.

In [203]:
comm_demand_charge_rates = pd.read_csv('data/demand_charge_monthly_peak_ratios.csv')
comm_demand_charge_rates['yearly_peak_demand_charge_per_mw'] = 5.36 * 0.96 * 1000

comm_demand_charge_rates['storage']=1
comm_demand_charge_rates = comm_demand_charge_rates[['lighting', 'hvac', 'refrigeration', 'storage', 'high_peak_demand_charge_per_mw',
       'low_peak_demand_charge_per_mw', 'yearly_peak_demand_charge_per_mw']]

In [204]:
#Calculate demand charge savings in each month based on yearly peak (for lighting, hvac, refrigeration). Yearly peak savings as a fraction of capacity are represented by the peak ratio in the hvac, refrig, and lighting columns.
comm_demand_charge_rates['lighting_yearly_peak_demand_charge_mw'] = 1*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']
comm_demand_charge_rates['hvac_yearly_peak_demand_charge_mw'] = 0.5*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']
comm_demand_charge_rates['refrigeration_yearly_peak_demand_charge_mw'] = 0.5*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']
comm_demand_charge_rates['storage_yearly_peak_demand_charge_mw'] = 1*comm_demand_charge_rates['yearly_peak_demand_charge_per_mw']

In [205]:
#Calculate demand charge savings in each month based on monthly high peak (for refrig and hvac).
comm_demand_charge_rates['refrigeration_high_peak_demand_charge_mw'] = comm_demand_charge_rates['refrigeration']*comm_demand_charge_rates['high_peak_demand_charge_per_mw']
comm_demand_charge_rates['hvac_high_peak_demand_charge_mw'] = comm_demand_charge_rates['hvac']*comm_demand_charge_rates['high_peak_demand_charge_per_mw']
comm_demand_charge_rates['storage_high_peak_demand_charge_mw'] = comm_demand_charge_rates['storage']*comm_demand_charge_rates['high_peak_demand_charge_per_mw']


In [206]:
#Calculate demand charge savings in each month based on monthly low peak (for refrig and hvac).
comm_demand_charge_rates['refrigeration_low_peak_demand_charge_mw'] = comm_demand_charge_rates['refrigeration']*comm_demand_charge_rates['low_peak_demand_charge_per_mw']
comm_demand_charge_rates['hvac_low_peak_demand_charge_mw'] = comm_demand_charge_rates['hvac']*comm_demand_charge_rates['low_peak_demand_charge_per_mw']
comm_demand_charge_rates['storage_low_peak_demand_charge_mw'] = comm_demand_charge_rates['storage']*comm_demand_charge_rates['low_peak_demand_charge_per_mw']


In [207]:
comm_demand_charge_rates

,lighting,hvac,refrigeration,storage,high_peak_demand_charge_per_mw,low_peak_demand_charge_per_mw,yearly_peak_demand_charge_per_mw,lighting_yearly_peak_demand_charge_mw,hvac_yearly_peak_demand_charge_mw,refrigeration_yearly_peak_demand_charge_mw,storage_yearly_peak_demand_charge_mw,refrigeration_high_peak_demand_charge_mw,hvac_high_peak_demand_charge_mw,storage_high_peak_demand_charge_mw,refrigeration_low_peak_demand_charge_mw,hvac_low_peak_demand_charge_mw,storage_low_peak_demand_charge_mw
0,1,0.19,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,866.4,4560,0.0,0.0,0
1,1,0.19,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,866.4,4560,0.0,0.0,0
2,1,0.22,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,1003.2,4560,0.0,0.0,0
3,1,0.26,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,1185.6,4560,0.0,0.0,0
4,1,0.32,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,1459.2,4560,0.0,0.0,0
5,1,0.38,0.5,1,9600,3600,5145.6,5145.6,2572.8,2572.8,5145.6,4800.0,3648.0,9600,1800.0,1368.0,3600
6,1,0.48,0.5,1,9600,3600,5145.6,5145.6,2572.8,2572.8,5145.6,4800.0,4608.0,9600,1800.0,1728.0,3600
7,1,0.50,0.5,1,9600,3600,5145.6,5145.6,2572.8,2572.8,5145.6,4800.0,4800.0,9600,1800.0,1800.0,3600
8,1,0.46,0.5,1,9600,3600,5145.6,5145.6,2572.8,2572.8,5145.6,4800.0,4416.0,9600,1800.0,1656.0,3600
9,1,0.38,0.5,1,4560,0,5145.6,5145.6,2572.8,2572.8,5145.6,2280.0,1732.8,4560,0.0,0.0,0


In [218]:
comm_demand_charge_savings = comm_demand_charge_rates.sum().reset_index()

In [219]:
comm_demand_charge_savings

,index,0
0,lighting,12.00
1,hvac,3.85
2,refrigeration,6.00
3,storage,12.00
4,high_peak_demand_charge_per_mw,74880.00
5,low_peak_demand_charge_per_mw,14400.00
6,yearly_peak_demand_charge_per_mw,61747.20
7,lighting_yearly_peak_demand_charge_mw,61747.20
8,hvac_yearly_peak_demand_charge_mw,30873.60
9,refrigeration_yearly_peak_demand_charge_mw,30873.60


In [220]:
comm_demand_charge_savings = comm_demand_charge_savings.iloc[7:]

In [221]:
comm_demand_charge_savings

,index,0
7,lighting_yearly_peak_demand_charge_mw,61747.2
8,hvac_yearly_peak_demand_charge_mw,30873.6
9,refrigeration_yearly_peak_demand_charge_mw,30873.6
10,storage_yearly_peak_demand_charge_mw,61747.2
11,refrigeration_high_peak_demand_charge_mw,37440.0
12,hvac_high_peak_demand_charge_mw,26728.8
13,storage_high_peak_demand_charge_mw,74880.0
14,refrigeration_low_peak_demand_charge_mw,7200.0
15,hvac_low_peak_demand_charge_mw,6552.0
16,storage_low_peak_demand_charge_mw,14400.0


In [222]:
comm_demand_charge_savings.rename(columns={'index':'resource_peakperiod', 0:'demand_charge_savings_per_mw'},inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [224]:
comm_demand_charge_savings.to_csv('data/annual_demand_charge_savings.csv')